In [1]:
import pandas as pd
import numpy as np
import PIL
from keras.applications import VGG16,imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import Model
from keras.layers import Dense
import numpy as np
import glob
import os

import pickle
import time


Using TensorFlow backend.


In [2]:
df = pd.read_table('Data/DeepFashion/list_attr_img.txt',skiprows=1,sep='\s+',header=None)

In [3]:
bbox = pd.read_table('Data/DeepFashion/list_bbox.txt',sep='\s+')

In [4]:
joined = bbox.join(df,lsuffix='image_name',rsuffix='0')

In [5]:
attr = (df.iloc[:,6:].values>0)
a_counts = (np.sum(attr,axis=0))
top_a = np.flip(a_counts.argsort(),0)[:206]
top_classes = np.where(np.sum(attr,axis=0) > 1000)[0]
attr_subset = attr[:,top_a]


In [ ]:
subset = pd.DataFrame([], columns=joined.columns)
for a in top_a[::-1]:
    needed = 1000 - np.sum(subset.iloc[:,6+a]>0)
    if needed > 0:
        has_attr = joined[joined.iloc[:,6+a]>0]
        sample = np.random.randint(has_attr.shape[0],size=needed)
        subset = subset.append(has_attr.iloc[sample])

In [ ]:
pickle.dump(subset,open('../rebalanced_attr_subset.p','wb'))

In [6]:
subset = pickle.load(open('../rebalanced_attr_subset.p','rb'))

In [7]:
attr_subset = (subset.iloc[:,6+top_a].values>0)

In [8]:
subset.shape

(85682, 1006)

In [ ]:
df = pickle.load(open('../df_subset/attr.p','rb'))

In [ ]:
df.shape

In [9]:
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input

def preprocess_DF(bbox):
    image = crop_resize_DF(bbox)
    image = img_to_array(image)
    image = np.expand_dims(image,axis=0)
    
    return preprocess(image)

def crop_resize_DF(bbox):
    img = PIL.Image.open('Data/DeepFashion/'+bbox['image_name'])
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    
    img = img.crop((lx,ly,ux,uy))
    img = img.resize(inputShape, PIL.Image.ANTIALIAS)
    return img

In [10]:
def generator(df, batch_size):
    while True:
        subset = np.random.randint(df.shape[0],size=batch_size)
        proc_imgs = []
        for i in subset:
            proc_imgs.append(np.squeeze(preprocess_DF(df.iloc[i])))
    
        attr = (df.iloc[subset,6+top_a].values>0)
        yield np.array(proc_imgs), attr

In [ ]:
joined.head()

In [11]:
attr = attr_subset

In [12]:
pos_weight = np.prod(attr.shape)/np.sum(attr)

In [13]:
pos_weight

221.41995860252149

In [14]:
class_weights = 1/(np.mean(attr,axis=0)*1000+1e-8)

In [15]:
import keras.backend as K
import tensorflow as tf

def weighted_sigmoid_loss(y_true,y_pred):
    return tf.nn.weighted_cross_entropy_with_logits(y_true,y_pred,pos_weight=pos_weight*5)

def H_pred(y_true, y_pred):
    return K.mean(K.sum(K.abs(y_true-y_pred),axis=1))



In [17]:
from keras import optimizers
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout,Flatten
l = 0.01

base_model = VGG16(weights='imagenet')
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output

x = Flatten(name='flatten')(x)
x = BatchNormalization()(x)
x = Dropout(rate=0.2)(x)

x = Dense(4096, activation='relu', name='fc1', kernel_initializer='glorot_normal',
         bias_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(l))(x)
x = BatchNormalization()(x)
x = Dropout(rate=0.2)(x)

predictions = Dense(len(top_a), activation='sigmoid',name='predictions', kernel_initializer='glorot_normal',
         bias_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(l))(x)

In [18]:
new_model = Model(inputs=base_model.input, outputs=predictions)




In [19]:
from keras import optimizers,metrics

new_model.compile(optimizer='adam', loss=weighted_sigmoid_loss, metrics=[H_pred,metrics.categorical_accuracy])

In [21]:
from keras import callbacks
import time
tbCB = callbacks.TensorBoard(log_dir="logs/attr-retrain{}".format(time.time()), histogram_freq=0, write_graph=False, write_images=False)
checkpointCB = callbacks.ModelCheckpoint("checkpoints/retrain{}".format(time.time()),period=10)

h = new_model.fit_generator(generator(subset,64),steps_per_epoch=50,epochs=80,verbose=1,
                            callbacks=[tbCB,checkpointCB],class_weight=class_weights)

Epoch 1/80
50/50 [==============================] - 188s - loss: 18.8721 - H_pred: 102.2031 - categorical_accuracy: 0.0156   
Epoch 2/80
50/50 [==============================] - 178s - loss: 4.2840 - H_pred: 100.3992 - categorical_accuracy: 0.0203   
Epoch 3/80
50/50 [==============================] - 178s - loss: 3.5448 - H_pred: 98.6228 - categorical_accuracy: 0.0359   
Epoch 4/80
50/50 [==============================] - 179s - loss: 3.4763 - H_pred: 95.3763 - categorical_accuracy: 0.0578   
Epoch 5/80
50/50 [==============================] - 180s - loss: 3.4564 - H_pred: 95.8727 - categorical_accuracy: 0.0213   
Epoch 6/80
50/50 [==============================] - 176s - loss: 3.4568 - H_pred: 92.6753 - categorical_accuracy: 0.0106   
Epoch 7/80
50/50 [==============================] - 180s - loss: 3.6954 - H_pred: 95.9628 - categorical_accuracy: 0.0194   
Epoch 8/80
50/50 [==============================] - 181s - loss: 3.5973 - H_pred: 94.4701 - categorical_accuracy: 0.0281   
Epoch

OSError: Unable to create file (Unable to open file: name = 'checkpoints/retrain1500912250.1376626', errno = 2, error message = 'no such file or directory', flags = 13, o_flags = 242)

In [ ]:
h = new_model.fit_generator(generator(joined,128),steps_per_epoch=50,epochs=80,verbose=1,
                            callbacks=[tbCB,checkpointCB],class_weight=class_weights)

In [ ]:
p = new_model.predict(proc_tops[:20])

In [ ]:
np.where(p[0,:]>0.5)

In [ ]:
np.where(p[1,:]>0.5)

In [ ]:
new_model.save_weights('Data/Trained Models/DF-Retrain 7-19.h5')

In [ ]:
new_model.layers[:19]

In [ ]:
new_model.load_weights('Data/Trained Models/DF-Retrain 7-19.h5')

In [ ]:
feat_model = Model(inputs=new_model.input,outputs=new_model.get_layer('fc1').output)

In [ ]:
base_model = VGG16(weights='imagenet')

feat_model2 = Model(inputs=base_model.input,outputs=base_model.get_layer('fc1').output)

In [ ]:
import pickle
tops = pickle.load(open('Data/feature_matrix/tops_10000_df.p','rb'))

In [ ]:
proc_tops = []
for index, row in tops.iloc[:100].iterrows():
    proc_tops.append(np.squeeze(preprocess_DF(row)))
proc_tops = np.array(proc_tops)

In [ ]:
start = time.time()
fc6_retrain_tops = feat_model.predict(proc_tops)
#pickle.dump(fc6_retrain_tops,open('Data/feature_matrix/fc6_retrain_tops.p','wb'))
print(f'{time.time()-start:.2f} s')

In [ ]:
p = feat_model.predict(proc_tops[:20])

In [ ]:
p

In [ ]:
p2 = feat_model2.predict(proc_tops[:20])

In [ ]:
p2

In [ ]:
plt.hist(new_model.get_layer('predictions').get_weights()[0][:,0])

In [ ]:
plt.hist(new_model.get_layer('fc1').get_weights()[0][:,0])

In [ ]:
l = list(generator(joined,4))

In [ ]:
for x,y in generator(joined,64):
    print(K.get_value(H_pred(new_model.predict(x),y)))
    

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.hist(np.sum(attr[:1000],axis=1))

In [ ]:
229000/64

In [ ]:
64*224*224